In [15]:
from known_boundary.GP import optimise,optimise_warp
from known_boundary.utlis import Trans_function, get_initial_points
from known_boundary.acquisition_function import EI_acquisition_opt,MES_acquisition_opt,Warped_TEI2_acquisition_opt,LCB_acquisition_opt
import numpy as np
import matplotlib.pyplot as plt
import GPy
import torch
import botorch
from botorch.test_functions import Ackley,Levy,Beale,Branin,Hartmann,Rosenbrock,Powell
from botorch.utils.transforms import unnormalize,normalize
import warnings
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

In [16]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm

In [20]:
def transform(y,fstar):
  y_transformed = np.sqrt(2*(fstar - y))
  return y_transformed



def ERM(X,dim,fstar,model,mean_temp): # X is a 2-dimensional array because we will use it in scipy.minimize

  #model_temp = copy.deepcopy(model)
  X = X.reshape(-1,dim)

  mean_g,var_g = model.predict(X,include_likelihood=False)
  mean_g = mean_g + mean_temp

  sigma_g = np.sqrt(var_g)
  
  sigma_g[sigma_g<10**(-12)]=10**(-12)

  mu_f = fstar - 1/2*mean_g**2
  sigma_f = mean_g**2 * sigma_g

  gamma = (fstar - mu_f)/sigma_f      
  out=sigma_f * norm.pdf(gamma) + (fstar-mu_f) * norm.cdf(gamma)

  return out.ravel()  #make the shape to be 1 dimensional



def ERM_acquisition_opt(model,bounds,fstar,mean_temp): #bound should an array of size dim*2
  dim = bounds.shape[0]
  opts ={'maxiter':50*dim,'maxfun':50*dim,'disp': False}

  restart_num = 5*dim+5 #*dim
  X_candidate = []
  AF_candidate = []

  for i in range(restart_num):
    init_X = np.random.uniform(bounds[:, 0], bounds[:, 1],size=(30*dim, dim))
    value_holder = ERM(init_X,dim,fstar,model,mean_temp)
      
    x0=init_X[np.argmin(value_holder)]

    res = minimize(lambda x: ERM(X=x,dim=dim,fstar=fstar,model=model,mean_temp=mean_temp),x0,
                                  bounds=bounds,method="L-BFGS-B",options=opts) #L-BFGS-B  nelder-mead(better for rough function) Powell

    X_temp =  res.x      
    AF_temp = ERM(X=np.array(X_temp).reshape(-1,1),dim=dim,fstar=fstar,model=model,mean_temp=mean_temp)
    
    X_candidate.append(X_temp)
    AF_candidate.append(AF_temp)

  X_next = X_candidate[np.argmin(AF_candidate)]

  return X_next,np.min(AF_candidate)

In [6]:
fun = Branin(negate=True)
bounds= fun.bounds #np.array([[-5., 10.],[0., 15.]])
dim = bounds.shape[0]
fstar = -0.397887
iter_num = 10*dim
n_init = 4*dim
standard_bounds=np.array([0.,1.]*dim).reshape(-1,2)

In [15]:
N = 1
total_record_ERM = []

for exp in range(N):

  print(exp)
  
  seed = exp


  X_BO = get_initial_points(bounds, n_init,device,dtype,seed=seed)
  Y_BO = fun(X_BO).reshape(-1,1)


  best_value_holder = []
  best_value_holder.append(float(Y_BO.max()))

  iter_num = 10*dim


  for iter in range(iter_num):

    print(iter)

    best_value = best_value_holder[-1]

    train_Y = (Y_BO - Y_BO.mean()) / Y_BO.std()
    train_X = normalize(X_BO, bounds)
    
    
    train_Y = train_Y.numpy()
    train_X = train_X.numpy()

    fstar_standard = (fstar - Y_BO.mean()) / Y_BO.std()
    fstar_standard = fstar_standard.item()
                


    train_Y_transform = transform(y=train_Y,fstar=fstar_standard)
    mean_temp = np.mean(train_Y_transform)
    
    res = optimise(train_X,(train_Y_transform-mean_temp))
    # print('lengthscale is: ',np.sqrt(res[0])) 
    # print('variance is: ',res[1])
    kernel = GPy.kern.RBF(input_dim=dim,lengthscale= np.sqrt(res[0]),variance=res[1]) 
    m = GPy.models.GPRegression(train_X, train_Y_transform-mean_temp,kernel)
    m.Gaussian_noise.variance.fix(10**(-5))

    
    standard_next_X,erm_value = ERM_acquisition_opt(m,bounds=standard_bounds,fstar=fstar_standard,mean_temp=mean_temp)
    X_next = unnormalize(torch.tensor(standard_next_X), bounds).reshape(-1,dim)     
    Y_next = fun(X_next).reshape(-1,1)

    # Append data
    X_BO = torch.cat((X_BO, X_next), dim=0)
    Y_BO = torch.cat((Y_BO, Y_next), dim=0)
    

    best_value = float(Y_BO.max())
    print('best y: ', best_value)
    
      
    best_value_holder.append(best_value)


  best_value_holder = np.array(best_value_holder)
  total_record_ERM.append(best_value_holder)

0
0
best y:  -3.5451968551073154
1
best y:  -3.5451968551073154
2
best y:  -2.742167673523447
3
best y:  -1.070681719191347
4
best y:  -0.8083589928152257
5
best y:  -0.6512305639227378
6
best y:  -0.6512305639227378
7
best y:  -0.6512305639227378
8
best y:  -0.6381326641698113
9
best y:  -0.40168099587902617
10
best y:  -0.40168099587902617
11
best y:  -0.40168099587902617
12
best y:  -0.40168099587902617
13
best y:  -0.4011681294471767
14
best y:  -0.3998592567192283
15
best y:  -0.3998592567192283
16
best y:  -0.3998063501173448
17
best y:  -0.39970935758814363
18
best y:  -0.39954017634635974
19
best y:  -0.3991773994015997


In [16]:
total_record_ERM

[array([-3.54519686, -3.54519686, -3.54519686, -2.74216767, -1.07068172,
        -0.80835899, -0.65123056, -0.65123056, -0.65123056, -0.63813266,
        -0.401681  , -0.401681  , -0.401681  , -0.401681  , -0.40116813,
        -0.39985926, -0.39985926, -0.39980635, -0.39970936, -0.39954018,
        -0.3991774 ])]

In [22]:
total_record_ERM

[array([-3.54519686, -3.54519686, -3.54519686, -2.74217091, -1.06941218,
        -0.80574844, -0.61161354, -0.61161354, -0.46675346, -0.46675346,
        -0.45973213, -0.45973213, -0.40124304, -0.40124304, -0.40124304,
        -0.40124304, -0.39842602, -0.39842602, -0.39842602, -0.39842602,
        -0.3983046 ])]

# new


In [18]:
def transform(y,fstar):
  y_transformed = np.sqrt(2*(y-fstar))
  return y_transformed



def ERM(X,dim,fstar,model,mean_temp): # X is a 2-dimensional array because we will use it in scipy.minimize

  #model_temp = copy.deepcopy(model)
  X = X.reshape(-1,dim)

  mean_g,var_g = model.predict(X,include_likelihood=False)
  mean_g = mean_g + mean_temp

  var_g[var_g<10**(-12)]=10**(-12)
  sigma_g = np.sqrt(var_g)
  

  mu_f = fstar + 1/2*mean_g**2
  sigma_f = mean_g**2 * sigma_g

  gamma = (mu_f - fstar)/sigma_f      
  out=sigma_f * norm.pdf(gamma) + (mu_f - fstar) * norm.cdf(gamma)

  return out.ravel()  #make the shape to be 1 dimensional



def ERM_acquisition_opt(model,bounds,fstar,mean_temp): #bound should an array of size dim*2
  dim = bounds.shape[0]
  opts ={'maxiter':50*dim,'maxfun':50*dim,'disp': False}

  restart_num = 3*dim
  X_candidate = []
  AF_candidate = []

  for i in range(restart_num):
    init_X = np.random.uniform(bounds[:, 0], bounds[:, 1],size=(30*dim, dim))
    value_holder = ERM(init_X,dim,fstar,model,mean_temp)
      
    x0=init_X[np.argmin(value_holder)]

    res = minimize(lambda x: ERM(X=x,dim=dim,fstar=fstar,model=model,mean_temp=mean_temp),x0,
                                  bounds=bounds,method="L-BFGS-B",options=opts) #L-BFGS-B  nelder-mead(better for rough function) Powell

    X_temp =  res.x      
    AF_temp = ERM(X=np.array(X_temp).reshape(-1,1),dim=dim,fstar=fstar,model=model,mean_temp=mean_temp)
    
    X_candidate.append(X_temp)
    AF_candidate.append(AF_temp)

  X_next = X_candidate[np.argmin(AF_candidate)]

  return X_next,np.min(AF_candidate)

In [19]:
fun = Branin(negate=False)
bounds= fun.bounds #np.array([[-5., 10.],[0., 15.]])
dim = bounds.shape[0]
fstar = 0.397887
iter_num = 10*dim
n_init = 4*dim
standard_bounds=np.array([0.,1.]*dim).reshape(-1,2)

fun = Trans_function(fun,fstar,min=True)

In [20]:
N = 1
total_record_ERM = []

for exp in range(N):

  print(exp)  
  seed = exp
  
  fstar0 = 0.
  Trans = False

  X_BO = get_initial_points(bounds, n_init,device,dtype,seed=seed)
  Y_BO = torch.tensor(
            [fun(x) for x in X_BO], dtype=dtype, device=device
        ).reshape(-1,1)
  
  #print(X_BO)

  best_value_holder = []
  best_value_holder.append(float(Y_BO.min()))

  iter_num = 10*dim


  for iter in range(iter_num):

    print(iter)

    best_value = best_value_holder[-1]

    train_Y = (Y_BO - Y_BO.mean()) / Y_BO.std()
    train_X = normalize(X_BO, bounds)
    
    train_Y = train_Y.numpy()
    train_X = train_X.numpy()
    

    fstar_standard = (fstar0 - Y_BO.mean()) / Y_BO.std()
    fstar_standard = fstar_standard.item()
    
    if not Trans:
      minimal = np.min(train_X)
      res = optimise(train_X,train_Y)
      kernel = GPy.kern.RBF(input_dim=dim,lengthscale= np.sqrt(res[0]),variance=res[1]) 
      m = GPy.models.GPRegression(train_X, train_Y,kernel)
      m.Gaussian_noise.variance.fix(10**(-5))

      standard_next_X = EI_acquisition_opt(m,bounds=standard_bounds,f_best=minimal)
      
      beta = np.sqrt(np.log(train_X.shape[0]))
      _,lcb = LCB_acquisition_opt(m,standard_bounds,beta)
      if lcb < fstar_standard:
        Trans = True
      
    else:
      
      print('transfromed GP')
              
      train_Y_transform = transform(y=train_Y,fstar=fstar_standard)
      mean_temp = np.mean(train_Y_transform)
      
      res = optimise(train_X,(train_Y_transform-mean_temp))
      # print('lengthscale is: ',np.sqrt(res[0])) 
      # print('variance is: ',res[1])
      kernel = GPy.kern.RBF(input_dim=dim,lengthscale= np.sqrt(res[0]),variance=res[1]) 
      m = GPy.models.GPRegression(train_X, train_Y_transform-mean_temp,kernel)
      m.Gaussian_noise.variance.fix(10**(-5))
      standard_next_X,erm_value = ERM_acquisition_opt(m,bounds=standard_bounds,fstar=fstar_standard,mean_temp=mean_temp)
    
    
    X_next = unnormalize(torch.tensor(standard_next_X), bounds).reshape(-1,dim)     
    Y_next = fun(X_next).reshape(-1,1)

    # Append data
    X_BO = torch.cat((X_BO, X_next), dim=0)
    Y_BO = torch.cat((Y_BO, Y_next), dim=0)
    

    best_value = float(Y_BO.min())
    print('best y: ', best_value)
    
      
    best_value_holder.append(best_value)


  best_value_holder = np.array(best_value_holder)
  total_record_ERM.append(best_value_holder)

0
0
best y:  3.1473098551073155
1
transfromed GP
best y:  3.1473098551073155
2
transfromed GP
best y:  2.481894221005147
3
transfromed GP
best y:  1.2211667342332477
4
transfromed GP
best y:  0.3265452929248366
5
transfromed GP
best y:  0.13018651228817724
6
transfromed GP
best y:  0.13018651228817724
7
transfromed GP
best y:  0.09177621358155619
8
transfromed GP
best y:  0.09177621358155619
9
transfromed GP
best y:  0.09177621358155619
10
transfromed GP
best y:  0.0013233276391514393
11
transfromed GP
best y:  0.0013233276391514393
12
transfromed GP
best y:  0.0013233276391514393
13
transfromed GP
best y:  0.00037164668093658815
14
transfromed GP
best y:  0.00037164668093658815
15
transfromed GP
best y:  0.00037164668093658815
16
transfromed GP
best y:  0.00028922567832168067
17
transfromed GP
best y:  0.00027766375618198946
18
transfromed GP
best y:  0.00027766375618198946
19
transfromed GP
best y:  0.00027766375618198946


In [8]:
total_record_ERM

[array([3.54519686, 3.54519686, 3.54519686, 2.82319108, 1.01874092,
        0.74498457, 0.74498457, 0.74498457, 0.74498457, 0.74498457,
        0.74498457, 0.74498457, 0.74498457, 0.74498457, 0.74498457,
        0.74498457, 0.74498457, 0.74498457, 0.74498457, 0.74498457,
        0.74498457])]